In [7]:
inventory = """{
  "sofa": {
    "price": 1000.0,
    "volume": 2.5
  },
  "television": {
    "price": 500.0,
    "volume": 0.02
  },
  "refrigerator": {
    "price": 1200.0,
    "volume": 0.8
  },
  "microwave": {
    "price": 150.0,
    "volume": 0.03
  },
  "bed": {
    "price": 800.0,
    "volume": 1.5
  },
  "desk": {
    "price": 200.0,
    "volume": 0.4
  },
  "chair": {
    "price": 50.0,
    "volume": 0.2
  },
  "lamp": {
    "price": 30.0,
    "volume": 0.01
  },
  "bookshelf": {
    "price": 100.0,
    "volume": 0.5
  },
  "computer": {
    "price": 1200.0,
    "volume": 0.02
  },
  "coffee table": {
    "price": 75.0,
    "volume": 0.2
  },
  "dining table": {
    "price": 250.0,
    "volume": 1.0
  },
  "wardrobe": {
    "price": 350.0,
    "volume": 1.5
  },
  "oven": {
    "price": 500.0,
    "volume": 0.3
  },
  "dishwasher": {
    "price": 400.0,
    "volume": 0.3
  },
  "washing machine": {
    "price": 600.0,
    "volume": 0.6
  },
  "toaster": {
    "price": 20.0,
    "volume": 0.01
  },
  "blender": {
    "price": 30.0,
    "volume": 0.02
  },
  "rug": {
    "price": 80.0,
    "volume": 0.1
  },
  "clock": {
    "price": 25.0,
    "volume": 0.01
  }
}"""

In [2]:
STORY = """In the heart of the apartment lies a room of quiet reflection and muted hues, whispering tales of homeliness
 and tranquility. The floors, washed in the warmth of polished oak, play host to an eclectic collection of cherished
  objects, each a beacon of the room's characteristic charm.

An emerald green sofa, its plush cushions like rolling hills of velvety softness, commands attention as it proudly sits
 in the center of the room. It is a monument of many an evening spent immersed in the pages of a gripping novel or in
 the laughter of shared stories. At its feet, a rug of rich burgundy, its fibers gently tickling bare feet, maps out the
  heart of this sanctuary.

To its right, a humble coffee table of natural wood takes center stage. On it, a small collection of coffee-stained mugs
 and well-thumbed novels narrate tales of quiet mornings and lazy afternoons. Above it, a lamp, its shade the color of
 fresh cream, casts a warm, inviting glow across the room, shadows dancing in its light.

Against one wall stands a bookshelf, an aged sentinel of oak and iron. It houses a world of adventures and knowledge,
 its shelves groaning under the weight of tomes old and new. Beside it, a chair, upholstered in soft, sky-blue fabric,
  invites the weary to rest.

In the corner, a lone desk of polished mahogany stands. A testament to late-night musings and feverish scribblings,
 its surface is littered with sheets of paper and an array of writing tools. Above it, a clock, its hands dancing in
  an endless waltz, keeps rhythm with the heartbeat of the room.

Every item, every piece of furniture, is a verse in the poem that is this room. They paint a portrait of serenity,
a world within a world, where each moment unfurls with the soft rustle of turning pages and the comforting whispers
of home."""

In [3]:
from langchain.tools import StructuredTool

def inventory_value_calculator(inventory_item_prices: list) -> float:
        """
        This function helps you calculate the sum of prices of multiple inventories given a list of prices.
        :param inventory_item_prices: List of inventory prices.
        :return: The sum of prices
        """
        room_inventory_value = sum(inventory_item_prices)
        return room_inventory_value

def inventory_value_calculator_tool():
    structured_tool = StructuredTool.from_function(
        name="calculate the value of the inventory in a room.",
        func=inventory_value_calculator,
        description="Use this tool if you want to calculate the total value of the inventory of a room.")

    return structured_tool

In [4]:
#from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames
parameters = {
    GenTextParamsMetaNames.DECODING_METHOD: "greedy",
    GenTextParamsMetaNames.MAX_NEW_TOKENS: 300,
    GenTextParamsMetaNames.MIN_NEW_TOKENS: 50,
    
}
project_id = ""
llm = WatsonxLLM(model_id="ibm/granite-13b-instruct-v2",url="https://us-south.ml.cloud.ibm.com",apikey="",project_id=project_id, params=parameters)

In [5]:
from langchain.agents import create_json_agent
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.tools.json.tool import JsonSpec

In [8]:
import json
data = json.loads(inventory)
print(type(data))

<class 'dict'>


In [9]:
def get_json_agent(data):
    #with open(json_path) as f:
    #    data = json.load(f)
    json_spec = JsonSpec(dict_=data, max_value_length=4000)
    json_toolkit = JsonToolkit(spec=json_spec)

    json_agent = create_json_agent(
        llm=llm,
        toolkit=json_toolkit,
        verbose=True
    )
    return json_agent

In [11]:
def prices_retrieval_tool():
    """
    This function will help you find the price of a single inventory.
    :return: the price
    """
    def get_price(inventory_item: str) -> str:
        json_agent = get_json_agent(data)
        result = json_agent_executor.run(
            f"""get the price of {inventory_item} from the json file.
            Find the closest match to the item you're looking for in that json, e.g.
             if you're looking for "mahogany oak table" and that is not in the json, use "table".
            Be mindful of the format of the json - there is no list that you can access via [0], so don't try to do that
            """)
        return result

    price_tool = StructuredTool.from_function(func=get_price,
                                              name='get inventory and furniture prices',
                                              description='This function will help you get the price of an inventory or'
                                                          ' furniture item.')

    return price_tool

In [12]:
tools = [inventory_value_calculator_tool(), prices_retrieval_tool()]
template = """
        You should read a text and find all the inventory items that are mentioned to be in a room.
        You have to print all the inventories that you find before getting the prices.
        you have to get all the prices before computing the total price. Be mindful that some items may appear several
        times and that you need to get a price for all items, so e.g. if five refrigerators are mentioned, you need the
        value of the refrigerator five times.
        Do not guess or estimate the price of the items or inventories. you must rely only on the prices that you get from the tools.
        Use those prices to calculate the total value of items in the text using the tools provided to you.
        Do not calculate the prices with anything else but the tools provided to you.


           Begin!

           Text: {input}
            """

In [13]:
from langchain.agents.initialize import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_core.prompts.prompt import PromptTemplate
agent = initialize_agent(llm=llm,
                         tools=tools,
                         agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)
prompt = PromptTemplate(input_variables=['input'], template=template)
agent.run(prompt.format(input=STORY))



> Entering new AgentExecutor chain...
Action: get inventory and furniture prices
inventory_item_prices: list
room_in_question: str

room_in_question = "room"
inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room_in_question)

inventory_item_prices = get_inventory_and_furniture_prices(room

> Finished chain.


'Action: get inventory and furniture prices\ninventory_item_prices: list\nroom_in_question: str\n\nroom_in_question = "room"\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room_in_question)\n\ninventory_item_prices = get_inventory_and_furniture_prices(room'